# 加速：多線程爬蟲



* 了解知乎 API 使用方式與回傳內容
* 撰寫程式存取 API 且添加標頭

## 作業目標

* 找一個之前實作過的爬蟲改用多線程改寫，比較前後時間的差異。





In [29]:
import requests
import re
from bs4 import BeautifulSoup
import os
import time
import threading

input_keyword = "e投睿"  # 這裡可以自己定義有興趣的關鍵字
utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

# 模擬封包的標頭
headers = {
    'authority': 'zh.wikipedia.org',
    'method': 'GET',
    'path': '/wiki/' + root_keyword_link,
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
    'dnt': '1',
    #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
    'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}    

url = 'https://zh.wikipedia.org' + root_keyword_link  # 組合關鍵字查詢URL

resp = requests.get(url, headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")
content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

#
# 解析回傳資料，並萃取文章內容
#
for paragraph in content:
    print(paragraph.get_text())


for ext_link in content:
    a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
    if len(a_tag) > 0:
        for link_string in a_tag:
            a_link = link_string["href"]       # 外部連結的網址
            a_keyword = link_string.get_text()  # 外部連結的中文名稱
            print("外部連結: [%s] %s" % (a_keyword, a_link))

class myThread (threading.Thread):
    def __init__(self, key_word_link, key_word, recursive):
        threading.Thread.__init__(self)
        self.key_word_link = key_word_link
        self.key_word = key_word
        self.recursive = recursive
    def run(self):
        WikiArticle(self.key_word_link, self.key_word, self.recursive, multithread=True)

def WikiArticle(key_word_link, key_word, recursive,multithread=False):
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #

        path = "./WikiArticle"
        if not os.path.isdir(path):
            os.mkdir(path)

        files = path+'/'+key_word+'.txt'

        with open(files, "w", encoding="UTF-8") as fh:
            for paragraph in content:
                fh.write(paragraph.get_text())

        
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #

        external_link_dict = dict({})
        for ext_link in content:
            a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
            if len(a_tag) > 0:
                for link_string in a_tag:
                    a_link = link_string["href"]       # 外部連結的網址
                    a_keyword = link_string.get_text()  # 外部連結的中文名稱
                    external_link_dict[a_link] = a_keyword 

        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        thread_list = []

        if (len(external_link_dict) > 0):       
            recursive = recursive + 1  # 遞迴深度加1
            for k, v in external_link_dict.items():
                if multithread:
                    thread = myThread(k, v, recursive)
                    thread_list.append(thread)
                    thread.start()
                else:
                    WikiArticle(k, v, recursive)  # 再次呼叫同樣的函數，執行同樣的流程
            if multithread:
                for thread in thread_list:
                    thread.join()

# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 1

startTime = time.time()
WikiArticle(root_keyword_link, input_keyword, 0, 0)
finishTime = time.time()
print('單線程所需時間:',finishTime - startTime) # 正常情況的爬蟲所需時間


e投睿: E%E6%8A%95%E7%9D%BF
/wiki/E%E6%8A%95%E7%9D%BF
e投睿（英語：eToro），是金融投資服務跟跟單交易服務（英語：Copy trading）的社交交易經紀公司，提供超過一千種的金融資產交易，分公司由歐洲賽普勒斯證券交易委員會（英語：Cyprus Securities and Exchange Commission）跟英國金融行為監管局（英語：Financial Conduct Authority）進行管制。

eToro是2007年成立的金融投資服務跟跟單交易服務（英語：Copy trading）的社交交易經紀公司，提供超過一千種的金融資產交易。eToro歐洲分公司在歐洲賽普勒斯註冊，公司編號200585[8]，受到歐洲賽普勒斯證券交易委員會（英語：Cyprus Securities and Exchange Commission）規管；英國分公司受到英國金融行為監管局（英語：Financial Conduct Authority）規管[9][1]；澳大利亞分公司受到澳洲證券投資委員會規管[10]。

2006年在以色列特拉維夫市成立RetailFX，並且在2007年改名eToro進入金融市場。[4][2][11]2016年4月4日，全國廣播公司商業頻道將eToro列入「值得關注的10家最熱門金融科技初創企業」名單。[12]2018年3月21日，中國民生金融、思佰益、韓投夥伴、國際投資公司聯合投資Etoro總計1億美元。[13]同年，eToro創建區塊鏈子公司eToroX，由多倫·羅森布拉姆（英語：Doron Rosenblum）擔任常務董事。[4]5月22日，eToro以加密貨幣進入美國金融市場。[14]2019年3月25日，eToro以未公開的價格收購Firmo[15]；同年11月14日，etoro以未公開的價格收購應用程式Delta。[4] 2019年3月25日eToro與終極格鬥冠軍賽(UFC)簽署為期六個月的讚助協議，在澳大利亞和紐西蘭市場推廣品牌。[16]

外部連結: [金融投資服務] /wiki/%E9%87%91%E8%9E%8D
外部連結: [社交交易] /wiki/%E7%A4%BE%E4%BA%A4%E4%BA%A4%E6%98%93
外部連結: [經紀公司] /wiki/%E7%B6%93

In [30]:
startTime = time.time()
WikiArticle(root_keyword_link, input_keyword, 0, multithread=True)
finishTime = time.time()
print('多線程所需時間:',finishTime - startTime) # 正常情況的爬蟲所需時間

遞迴層[0] - /wiki/E%E6%8A%95%E7%9D%BF (e投睿)
遞迴層[1] - /wiki/%E9%87%91%E8%9E%8D (金融投資服務)
遞迴層[1] - /wiki/%E7%A4%BE%E4%BA%A4%E4%BA%A4%E6%98%93 (社交交易)
遞迴層[1] - /wiki/%E7%B6%93%E7%B4%80%E5%85%AC%E5%8F%B8 (經紀公司)
遞迴層[1] - /wiki/%E6%AD%90%E6%B4%B2 (歐洲)
遞迴層[1] - /wiki/%E8%B3%BD%E6%99%AE%E5%8B%92%E6%96%AF (賽普勒斯)
遞迴層[1] - /wiki/%E4%BB%A5%E8%89%B2%E5%88%97 (以色列)
遞迴層[1] - /wiki/%E7%89%B9%E6%8B%89%E7%BB%B4%E5%A4%AB (特拉維夫市)
遞迴層[1] - /wiki/%E9%87%91%E8%9E%8D%E5%B8%82%E5%A0%B4 (金融市場)
遞迴層[1] - /wiki/%E5%85%A8%E5%9B%BD%E5%B9%BF%E6%92%AD%E5%85%AC%E5%8F%B8%E5%95%86%E4%B8%9A%E9%A2%91%E9%81%93 (全國廣播公司商業頻道)
遞迴層[1] - /wiki/%E4%B8%AD%E5%9C%8B%E6%B0%91%E7%94%9F%E9%87%91%E8%9E%8D (中國民生金融)
遞迴層[1] - /wiki/%E6%80%9D%E4%BD%B0%E7%9B%8A (思佰益)
遞迴層[1] - /wiki/%E5%8C%BA%E5%9D%97%E9%93%BE (區塊鏈)
遞迴層[1] - /wiki/%E5%B8%B8%E5%8A%A1%E8%91%A3%E4%BA%8B (常務董事)
遞迴層[1] - /wiki/%E5%8A%A0%E5%AF%86%E8%B2%A8%E5%B9%A3 (加密貨幣)
遞迴層[1] - /wiki/%E7%BE%8E%E5%9C%8B (美國)
遞迴層[1] - /wiki/%E6%87%89%E7%94%A8%E7%A8%8B%E5%BC%8F (應用程式)
遞迴層[1] - /wiki/%E7%B